<a href="https://colab.research.google.com/github/Sakha-Language-Processing/wordforms/blob/main/verbs_at_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Автоматизированный поиск глаголов как последних слов в предложении



Из 541024 предложений выделены последние слова, которые с большой вероятностью являются глаголами. Получилось 20408 словоформ. Доля глаголов является высокой. 

## Загрузка и очистка 

In [1]:
# !wget https://github.com/Sakha-Language-Processing/sakha-parser-keskil/blob/main/raw_data_keskil_fixed.zip
# !unzip raw_data_keskil_fixed.zip
# parlament
!wget https://github.com/Sakha-Language-Processing/parser-palament/raw/main/raw_data_parlamentsakha.zip
!unzip raw_data_parlamentsakha.zip
!wget https://github.com/Sakha-Language-Processing/parser-nvkonline/raw/main/raw_data_nvkonline.zip
!unzip raw_data_nvkonline.zip
!wget https://github.com/Sakha-Language-Processing/sakha-parser-keskil/raw/main/raw_data_keskil_fixed.zip
!unzip raw_data_keskil_fixed.zip
!wget https://github.com/Sakha-Language-Processing/parser-kyym/raw/main/kyym-df.zip
!unzip kyym-df.zip
!wget https://github.com/Sakha-Language-Processing/parser-ysia/raw/main/sakha_ysia_df.zip
!unzip sakha_ysia_df.zip

--2021-04-18 08:22:01--  https://github.com/Sakha-Language-Processing/parser-palament/raw/main/raw_data_parlamentsakha.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/Sakha-Language-Processing/parser-parlament/raw/main/raw_data_parlamentsakha.zip [following]
--2021-04-18 08:22:01--  https://github.com/Sakha-Language-Processing/parser-parlament/raw/main/raw_data_parlamentsakha.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Sakha-Language-Processing/parser-parlament/main/raw_data_parlamentsakha.zip [following]
--2021-04-18 08:22:01--  https://raw.githubusercontent.com/Sakha-Language-Processing/parser-parlament/main/raw_data_parlamentsakha.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 18

In [13]:
# Чтение файлов
# Можно добавить свои список файлов 
import json
import pandas as pd

articles = []
for file in ['raw_data_nvkonline.json',
             'raw_data_parlamentsakha.json',
             'raw_data_keskil.json']:
    file_ = open(file) 
    articles  += json.load(file_)
    file_.close()

all_files_text = []
for article in articles:
    all_files_text += article['raw_text'] 
    if 'title' in article:
        all_files_text += [article['title']]

aytal_text = ' '
for file in ['kyym-novosti.csv',
             'sakha_ysia.csv']:
    file_ = pd.read_csv(file) 
    aytal_text  += ' '.join(list(file_['title'].fillna(' ')+file_['all_text'].fillna(' ')))

all_files_text = ' '.join(all_files_text) + ' '.join(aytal_text)
del aytal_text

In [131]:
import re
all_files_text = re.sub(re.compile(r'https?\S+'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'<.*?>'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'@\S+'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'[\//:@&\-\'\`\"\_\n\#]'), ' ', all_files_text)
all_files_text = re.sub(re.compile(r'\s*[A-Za-z\d.]+\b'), ' ' , all_files_text)
all_files_text = all_files_text.lower()
similar_characters = {'h':'һ','c':'с','x':'х','y':'у',
                      'e':'е', 'T':'Т', 'o':'о', 'p':'р', 
                      'a':'а', 'k':'к', 'b':'в', 'm':'м', 
                      '5':'ҕ', '8':'ө'}
for loaned, ersatz in similar_characters.items():
    all_files_text = all_files_text.replace(loaned, ersatz)  



## Выделение последних слов предложений

In [142]:

all_files_text = all_files_text.replace('?', '.')
all_files_text = all_files_text.replace('!', '.')
regex = re.compile('[^а-яА-ЯһөҕүҥҺӨҔҮҤ. ]')
all_files_text = re.sub(regex, ' ', all_files_text)

sentence_tails = [item[-min(len(item),30):] for item in all_files_text.split('.')]

verbs = []
for item in sentence_tails:
    verb = item.split(' ')[-1]
    if len(verb)<1:
         for i in range(30):
            item = item.replace(' .','.')
            verb = item.split(' ')[-1]
    verbs.append(verb) 
verbs = sorted(list(set(verbs)))
df = pd.DataFrame(verbs)
df.columns = ['verb']
df.to_csv('verbs.csv',index=False)

In [141]:
len(sentence_tails)


541024